In [ ]:
# specify the path and then load packages
import sys
sys.path.insert(0, '/Users/timtamothy/Documents/GitHub/adsi_at2/')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data.sets import save_sets